In [1]:
import cv2
import requests 
import uuid
import os
import time
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
# shouls be stored in databricks secrets backed by Azure Databrick secrets
# 
# replace with your subscription key for face api 
# subscription_key = '3a991cb25c2347db81db939c957ce715'
# premium key 
subscription_key = '90773c9afa3d4b2195059d2a0f16e722'
assert subscription_key


# replace below with the correct location of face api account (need to test with containerized version)
face_api_url = 'https://eastus2.api.cognitive.microsoft.com/face/v1.0/detect?returnFaceLandmarks=true'
headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

# adjust to your parameter set     
params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,headPose,smile,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}

In [2]:
%run "./PyTorch Emotion Eval"

In [3]:
def process_frame_by_face_api(frame):
    headers = { 'Content-Type':'application/octet-stream','Ocp-Apim-Subscription-Key': subscription_key }
    ret, buf = cv2.imencode('.jpg', frame)
    response = requests.post(face_api_url, params=params, headers=headers, data=buf.tobytes())
    res = response.json()
    return res

In [4]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *
from pyspark.sql import *
import pandas as pd 
from pandas.io.json import json_normalize

def persistFaceToTable(faces, videoId, tableName):
  dfa = None
  print( len(faces))
  count = 0
  for faceResult in faces:
     flat_record = flatten_json(faceResult[2])
     b = json_normalize(flat_record)
     if len(flat_record) != 98:
       print(len(flat_record))
       print('odd sized reocrd, skipping')
       continue
     b = b.assign(frameId=str(faceResult[0]))
     b = b.assign(frameNum=faceResult[1])
     b = b.assign(videoId=videoId)
     torchVals = faceResult[3].data.tolist()
     b = b.assign(torchAnger=torchVals[0])
     b = b.assign(torchDisgust=torchVals[1])
     b = b.assign(torchFear=torchVals[2])
     b = b.assign(torchHappy=torchVals[3])
     b = b.assign(torchSad=torchVals[4])
     b = b.assign(torchSurprise=torchVals[5])
     b = b.assign(torchNeutral=torchVals[6])
     df = spark.createDataFrame(b)
     count = count + 1
     if dfa == None:
       dfa = df
     else:
       dfa = dfa.union(df)
  dfa.write.mode("append").saveAsTable(tableName)
  dfa = None
  

In [5]:
def process_sample_frames(video_filename, sample, videoId):
    """Extract frames from video"""
    cap = cv2.VideoCapture(video_filename)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    faces = []
    persist_count = 20
    
    print('video length/frame count: ', video_length)
    if cap.isOpened() and video_length > 0:
        success, image = cap.read()
        count = 0
        collectCount = 0
        while success:
            if count % sample == 0:
                """ process """
                face_result = process_frame_by_face_api(image) 
                # print(len(face_result))
                faces_found = len(face_result)
                for i in (range(0,faces_found)):
                    newId = uuid.uuid4()
                    x = face_result[i]['faceRectangle']['left']
                    y= face_result[i]['faceRectangle']['top']
                    h = face_result[i]['faceRectangle']['height']
                    w = face_result[i]['faceRectangle']['width']
                    crop_img = crop_img = image[y:y+h, x:x+w]
                    scores = eval_pyTorch_emotion(crop_img)
                    id = face_result[i]['faceId']
                    val = (newId, count, face_result[i], scores)
                    faces.append(val)                    
                collectCount += 1
                # so not to go over 10 frame requests per sec
                if collectCount % 8 == 0:
                  time.sleep(1.0)
            if len(faces) >= persist_count and len(faces) != 0:
              persistFaceToTable(faces, videoId, "tempFaceTable")
              faces = []
            success, image = cap.read()
            count += 1
        if len(faces) > 0:
          persistFaceToTable(faces, videoId, "tempFaceTable")
          faces = []
    return faces

In [6]:
#results = process_sample_frames('/dbfs/mnt/bs/IncomingVideos/making_face.mp4', 20)
#print (results)

In [7]:
#print(len(results))


                   